In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine

# 2016 Primary Results by County

In [2]:
#Store cvs file path in a variable
csv_e2016 = "../Resources/primary_results.csv"

#Read CSV into a Pandas dataframe
E2016_df = pd.read_csv(csv_e2016)

#Edit df to include only state, county, party and votes columns
primary2016_df = E2016_df[['state','county','party','votes']].copy()
primary2016_df = primary2016_df.rename(columns={'county':'ga_county'})

#Edit df to only include GA counties
GA2016_df = primary2016_df.loc[(primary2016_df['state']=="Georgia")]

#Set up separate dfs for each party
REP_2016_df = GA2016_df.loc[(GA2016_df['party'] == 'Republican')]
DEM_2016_df = GA2016_df.loc[(GA2016_df['party'] == 'Democrat')]

#Merge dfs to create 1 with both party data
Party_2016_df = pd.merge(REP_2016_df, DEM_2016_df, how = 'outer', on='ga_county')

#Remove unnecessary and rename resulting columns
Party_2016_df = Party_2016_df[['ga_county', 'party_x', 'votes_x', 'party_y', 'votes_y']]
Party_2016_df = Party_2016_df.rename(columns = {'party_x': 'party', 'votes_x':'republican_votes', 'party_y':'party_', 'votes_y':'democrat_votes'})

#Remove "county" and make all capital to match Race data tables
Party_2016_df['ga_county'] = Party_2016_df['ga_county'].str.split(" ",expand = True)
Party_2016_df['ga_county'] = Party_2016_df['ga_county'].str.upper()

##Party_2016_df includes columns indicating party, as well as rows for each (possibly) precint within a county

#Aggregate County Data - total votes per party per county
county_counts = Party_2016_df.groupby(['ga_county'])
county_counts_df = county_counts.sum()

#Add column to show total votes for each county
county_counts_df['total_votes'] = county_counts_df['republican_votes'] + county_counts_df['democrat_votes']
county_counts_df

,republican_votes,democrat_votes,total_votes
ga_county,,,
APPLING,6106,3160,9266
ATKINSON,1868,1250,3118
BACON,3316,855,4171
BAKER,848,1755,2603
BALDWIN,8888,18980,27868
...,...,...,...
WHITFIELD,25196,14215,39411
WILCOX,2150,1715,3865
WILKES,2976,4215,7191


# 2020 Election Results by County

In [3]:
#Store cvs file path in a variable
csv_e2020 = "../Resources/president_county_candidate.csv"

#Read CSV into a Pandas dataframe
E2020_df = pd.read_csv(csv_e2020)

#Edit df to rename columns
E2020_df = E2020_df.rename(columns={'county':'ga_county'})

#Edit to only include GA counties
GA2020_df = E2020_df.loc[(E2020_df['state']=="Georgia")]

#Create new dfs to show only Democrat and Republican party results
GA_REP_df = GA2020_df.loc[(GA2020_df['party'] == 'REP')]
GA_DEM_df = GA2020_df.loc[(GA2020_df['party'] == 'DEM')]

#Merge DEM/REP Dataframes
GA_2020_df = pd.merge(GA_REP_df, GA_DEM_df, how='outer', on='ga_county')

#Remove and rename resulting columns
GA_party_df = GA_2020_df[['ga_county', 'candidate_x', 'total_votes_x', 'candidate_y', 'total_votes_y']]
GA_party_df = GA_party_df.rename(columns = {'candidate_x': 'republican', 'total_votes_x':'rep_votes', 'candidate_y':'democrat', 'total_votes_y':'dem_votes'})

#Add column with total votes for each county
GA_party_df['total_votes'] = GA_party_df['rep_votes'] + GA_party_df['dem_votes']

#Remove "county" and make all capital to match Race data tables
GA_party_df['ga_county'] = GA_party_df['ga_county'].str.split(" ",expand = True)
GA_party_df['ga_county'] = GA_party_df['ga_county'].str.upper()
GA_party_df

,ga_county,republican,rep_votes,democrat,dem_votes,total_votes
0,APPLING,Donald Trump,6526,Joe Biden,1779,8305
1,ATKINSON,Donald Trump,2300,Joe Biden,825,3125
2,BACON,Donald Trump,4018,Joe Biden,625,4643
3,BAKER,Donald Trump,897,Joe Biden,652,1549
4,BALDWIN,Donald Trump,8903,Joe Biden,9140,18043
...,...,...,...,...,...,...
154,WHITFIELD,Donald Trump,25636,Joe Biden,10670,36306
155,WILCOX,Donald Trump,2403,Joe Biden,862,3265
156,WILKES,Donald Trump,2823,Joe Biden,2160,4983
157,WILKINSON,Donald Trump,2664,Joe Biden,2075,4739


## Load to Postgres

In [15]:
#Connect to local database
rds_connection_string = "postgres:postgres@localhost:5432/ga_election_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

#Check for tables
engine.table_names()

#Use pandas to load 2020 DataFrame into database
GA_party_df.to_sql(name='election_2020_ga', con=engine, if_exists='append', index=False)

#Use pandas to load 2016 DataFrame into database
county_counts_df.to_sql(name='primary_2016_ga', con=engine, if_exists='append', index=True)

['election_2020_ga',
 'primary_2016_ga',
 'race_data_2016',
 'fulton_trump_2020',
 'fulton_biden_2020',
 'race_data_2020']

## Confirm that data loaded into Postgres

In [9]:
pd.read_sql_query('select * from primary_2016_ga', con=engine).head()

,ga_county,republican_votes,democrat_votes,total_votes
0,APPLING,6106,3160,9266
1,ATKINSON,1868,1250,3118
2,BACON,3316,855,4171
3,BAKER,848,1755,2603
4,BALDWIN,8888,18980,27868


In [10]:
pd.read_sql_query('select * from election_2020_ga', con=engine).head()

,ga_county,republican,rep_votes,democrat,dem_votes,total_votes
0,APPLING,Donald Trump,6526,Joe Biden,1779,8305
1,ATKINSON,Donald Trump,2300,Joe Biden,825,3125
2,BACON,Donald Trump,4018,Joe Biden,625,4643
3,BAKER,Donald Trump,897,Joe Biden,652,1549
4,BALDWIN,Donald Trump,8903,Joe Biden,9140,18043


## Sample Query

In [7]:
#Import query dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import func

# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()
session = Session(engine)

In [17]:
E2020 = Base.classes.election_2020_ga
Race2020 = Base.classes.race_data_2020

session.query(E2020.ga_county, E2020.total_votes, Race2020.total_voters).\
    filter(E2020.ga_county == Race2020.ga_county).all()




[('APPLING', 8305, 11440),
 ('ATKINSON', 3125, 4801),
 ('BACON', 4643, 6726),
 ('BAKER', 1549, 2258),
 ('BALDWIN', 18043, 26742),
 ('BANKS', 8727, 12905),
 ('BARROW', 37257, 55694),
 ('BARTOW', 49766, 74380),
 ('BERRIEN', 7688, 11317),
 ('BIBB', 70053, 108126),
 ('BLECKLEY', 5639, 7558),
 ('BRANTLEY', 7690, 11302),
 ('BROOKS', 7050, 11170),
 ('BRYAN', 20983, 30651),
 ('BULLOCH', 29629, 44737),
 ('BURKE', 10609, 16570),
 ('BUTTS', 11680, 17195),
 ('CALHOUN', 2183, 3187),
 ('CAMDEN', 23218, 35141),
 ('CANDLER', 4402, 6328),
 ('CARROLL', 53714, 83866),
 ('CATOOSA', 32099, 45785),
 ('CHARLTON', 4522, 6562),
 ('CHATHAM', 131491, 202999),
 ('CHATTAHOOCHEE', 1547, 3943),
 ('CHATTOOGA', 9918, 14183),
 ('CHEROKEE', 142381, 190605),
 ('CLARKE', 50494, 76848),
 ('CLAY', 1427, 2053),
 ('CLAYTON', 111289, 194338),
 ('CLINCH', 2852, 4345),
 ('COBB', 387305, 537659),
 ('COFFEE', 15089, 25165),
 ('COLQUITT', 15964, 25087),
 ('COLUMBIA', 79249, 107656),
 ('COOK', 6959, 10650),
 ('COWETA', 75711, 102262